In [4]:
import sys
import os
from prefect import task, Flow

abspath = os.path.dirname(os.path.normpath(os.path.abspath(os.path.dirname(''))))

sys.path.append(abspath)

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

from toolbox.scrapers import *

# Data Structure: ['HCPCS/ CPT/NDC Code', 'Internal Proc Code', 'Other Code',
#        'Procedure Name', 'Charge', 'Medicare Adv - Aetna',
#        'Medicare Adv - BCBS', 'Medicare Adv - Humana', 'Medicare Adv - United',
#        'Medicare Adv - Other', 'Aetna', 'BCBS', 'Cigna', 'Exchange/Ambetter',
#        'Medcost', 'United', 'Other Mgd Care', 'Tricare', 'Workers Comp',
#        'Self Pay', 'De-identified Minimum', 'De-identified Maximum']


In [2]:
hospital_urls = get_source_urls(driver_path, abspath)


In [ ]:
df_cone = get_cone('cone-health', hospital_urls, raw_download_path)


In [12]:
df_duke_cdm, df_duke_drg = get_duke('duke-university-hospital', hospital_urls, raw_download_path)


In [6]:
df_ncb = get_north_carolina_baptist('north-carolina-baptist-hospital', hospital_urls, raw_download_path)


North-Carolina-Baptist-Hospital--Transparency-File-FINAL.csv?la=en
Davie-Medical-Center-Transparency-12312020-FINAL.csv?la=en
High-Point-Regional-Medical-Center-Transparency-File-12312020-FINAL.csv?la=en
Lexington-Medical-Center-Transparency-12312020-FINAL.csv?la=en
Wilkes-Regional-Medical-Center-Transparency-12312020-FINAL.csv?la=en


In [ ]:
df_ncb.head(10)

In [10]:
df_cdm_app, df_drg_app, df_shop_app = get_app('app-regional-health-system', hospital_urls, raw_download_path)


In [ ]:

df_catawba = get_catawba('catawba-valley-medical-center', hospital_urls,  raw_download_path)

df_comp_cateret, df_desc_cateret = get_cateret('cateret-health-care', hospital_urls,  raw_download_path)

df_mission = get_mission('mission-health', hospital_urls, raw_download_path)

df_novant = get_novant('novant-health', hospital_urls, raw_download_path)

df_nhrmc_op, df_nhrmc_ip = get_nhrmc('nhrmc-health', hospital_urls, raw_download_path)

df_northern = get_northern('northern-regional', hospital_urls, raw_download_path)

df_first_moore =  get_first('first-health-moore', hospital_urls, raw_download_path)

df_first_montgomery =  get_first('first-health-montgomery', hospital_urls, raw_download_path)

df_cdm_iredell, df_drg_iredell, df_drg_internet =  get_iredell('iredell-health', hospital_urls, raw_download_path)

df_wakemed_cary = get_wakemed('wakemed-cary', hospital_urls, raw_download_path, driver_path)

df_wakemed_raleigh = get_wakemed('wakemed-raleigh', hospital_urls, raw_download_path, driver_path)